- Task: Binary classification (≤ 50K vs. > 50K).

- Features: Mix of categorical (e.g., workclass, education, occupation)
and numeric (e.g., age, hours-per-week).

- Target: Income.

In [1]:
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo
adult = fetch_ucirepo(id=2)
X = adult.data.features # features (pandas DataFrame)
y = adult.data.targets # target (pandas DataFrame)

In [2]:
X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [3]:
y.head()

,income
0,<=50K
1,<=50K
2,<=50K
3,<=50K
4,<=50K


## Data Preparation

1. handle missing values (drop or impute)

In [4]:
X.isnull().sum()

,0
age,0
workclass,963
fnlwgt,0
education,0
education-num,0
marital-status,0
occupation,966
relationship,0
race,0
sex,0


In [5]:
X = X.dropna()

In [6]:
import pandas as pd
import numpy as np

# Select the 'income' column to get a Series
y = y['income']

# Align with X after dropping missing values
y = y.loc[X.index]

In [7]:
print(X.isna().sum())
X.head()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
dtype: int64


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


2. Encode categorical variables into numeric values (e.g., label encoding).

In [8]:
cat_cols = X.select_dtypes(include=['object']).columns
cat_cols

Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object')

In [9]:
from sklearn.preprocessing import LabelEncoder

encoded_df = X.copy()

le = LabelEncoder()

for col in cat_cols:
    encoded_df[col] = le.fit_transform(encoded_df[col].astype(str))

encoded_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5


In [10]:
encoded_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47621 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   age             47621 non-null  int64
 1   workclass       47621 non-null  int64
 2   fnlwgt          47621 non-null  int64
 3   education       47621 non-null  int64
 4   education-num   47621 non-null  int64
 5   marital-status  47621 non-null  int64
 6   occupation      47621 non-null  int64
 7   relationship    47621 non-null  int64
 8   race            47621 non-null  int64
 9   sex             47621 non-null  int64
 10  capital-gain    47621 non-null  int64
 11  capital-loss    47621 non-null  int64
 12  hours-per-week  47621 non-null  int64
 13  native-country  47621 non-null  int64
dtypes: int64(14)
memory usage: 5.4 MB


Split the dataset as:

80% training

20% validation

20% test

In [11]:
from sklearn.model_selection import train_test_split

X = encoded_df
y = y.values.ravel()

y_le = LabelEncoder()
y = y_le.fit_transform(y)


In [12]:

X_train, X_temp, y_train, y_temp = train_test_split(
    encoded_df, y, test_size=0.4, random_state=42, stratify=y
)

# Split temp into validation and test (each 20% of total)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)


## Build decision tree from scratch

### Implement a tree recursively.

At each split:
1. Compute both Gini Impurity and Entropy.
2. For each feature and split, calculate the weighted impurity of child
nodes.

3. Choose the split with the highest information gain (lowest impu-
rity).



Continue splitting until:


- All samples in a node have the same label, OR
- Maximum depth is reached, OR
- No further improvement in impurity.


Implement a function to predict for new samples.

In [13]:
import numpy as np

def gini(y):
  classes, counts = np.unique(y, return_counts = True)
  probs = counts/ counts.sum()
  return 1 - (probs**2).sum()

def entropy(y):
  classes, counts = np.unique(y, return_counts = True)
  probs = counts/ counts.sum()
  return - (probs * np.log2(probs)).sum()

In [14]:
def weighted_impurity(left_y, right_y, criterion='gini'):
  n = len(left_y) + len(right_y)
  if criterion == 'gini':
    imp = (len(left_y)/n)*gini(left_y) + (len(right_y)/n)*gini(right_y)
  elif criterion == 'entropy':
    imp = (len(left_y)/n)*entropy(left_y) + (len(right_y)/n)*entropy(right_y)
  else:
    raise ValueError("Invalid Criterion")
  return imp

In [15]:
class TreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value


In [16]:
def build_tree(X, y, depth=0, max_depth=5, criterion='gini'):

    if len(np.unique(y)) == 1:
        return TreeNode(value=y[0])


    if (max_depth is not None and depth >= max_depth) or len(y) <= 1:
        unique, counts = np.unique(y, return_counts=True)
        return TreeNode(value=unique[np.argmax(counts)])

    n_samples, n_features = X.shape
    best_impurity = float('inf')
    best_feature, best_threshold = None, None
    best_left_idx, best_right_idx = None, None


    for feature in range(n_features):
        thresholds = np.unique(X[:, feature])
        for t in thresholds:
            left_idx = X[:, feature] <= t
            right_idx = X[:, feature] > t
            if sum(left_idx) == 0 or sum(right_idx) == 0:
                continue
            imp = weighted_impurity(y[left_idx], y[right_idx], criterion)
            if imp < best_impurity:
                best_impurity = imp
                best_feature = feature
                best_threshold = t
                best_left_idx = left_idx
                best_right_idx = right_idx


    if best_feature is None:
        unique, counts = np.unique(y, return_counts=True)
        return TreeNode(value=unique[np.argmax(counts)])


    left_child = build_tree(X[best_left_idx], y[best_left_idx], depth+1, max_depth, criterion)
    right_child = build_tree(X[best_right_idx], y[best_right_idx], depth+1, max_depth, criterion)

    return TreeNode(feature=best_feature, threshold=best_threshold, left=left_child, right=right_child)


In [17]:
def predict_tree(node, X):
    preds = []
    for x in X:
        curr = node
        while curr.value is None:
            if x[curr.feature] <= curr.threshold:
                curr = curr.left
            else:
                curr = curr.right
        preds.append(curr.value)
    return np.array(preds)


## Pre-Pruning (Restricting Tree Growth)
While building the tree:
- Limit maximum depth (try depths = 2, 4, 6, and unlimited).
- Require at least a minimum number of samples (e.g., 5) to split.

In [18]:
from sklearn.metrics import accuracy_score

In [19]:

tree = build_tree(X_train.values, y_train, max_depth=2, criterion='entropy')
y_pred = predict_tree(tree, X_test.values)


print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.5639895013123359


In [20]:

tree = build_tree(X_train.values, y_train, max_depth=4, criterion='entropy')
y_pred = predict_tree(tree, X_test.values)


print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.5746981627296588


In [21]:

tree = build_tree(X_train.values, y_train, max_depth=6, criterion='entropy')
y_pred = predict_tree(tree, X_test.values)


print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.577742782152231


## Post-Pruning (Reduced Error Pruning)
1. First grow a full tree.
2. Then, for each internal node:
    - Replace it with a leaf (majority class).
    - Check validation accuracy.
3. If accuracy does not decrease, keep the pruning.
4. Repeat until no further improvement.

In [22]:
def majority_class(y):
    unique, counts = np.unique(y, return_counts=True)
    return unique[np.argmax(counts)]


In [23]:
def post_prune(node, X_val, y_val):
    # Base case: if leaf, do nothing
    if node.value is not None:
        return

    # Recursively prune left and right subtrees first
    post_prune(node.left, X_val, y_val)
    post_prune(node.right, X_val, y_val)

    # Try pruning this node: replace with leaf
    # Save original children
    left_backup = node.left
    right_backup = node.right
    original_feature = node.feature
    original_threshold = node.threshold

    # Temporarily make this node a leaf
    node.left = None
    node.right = None
    node.feature = None
    node.threshold = None

    # Assign majority class from training samples at this node
    # NOTE: You need to know which samples reach this node
    # For simplicity, assume node has a `samples_y` attribute storing training labels
    node.value = majority_class(node.samples_y)

    # Evaluate validation accuracy
    y_pred = predict_tree(node, X_val.values)
    acc_pruned = np.mean(y_pred == y_val)

    # Evaluate accuracy with original subtree
    node.left = left_backup
    node.right = right_backup
    node.feature = original_feature
    node.threshold = original_threshold
    node.value = None
    y_pred_orig = predict_tree(node, X_val.values)
    acc_orig = np.mean(y_pred_orig == y_val)

    # Keep pruning if accuracy does not decrease
    if acc_pruned >= acc_orig:
        node.left = None
        node.right = None
        node.feature = None
        node.threshold = None
        node.value = majority_class(node.samples_y)


In [24]:

full_tree = build_tree(X_train.values, y_train, max_depth=None, criterion='entropy')


def store_samples(node, X, y):
    node.samples_y = y
    if node.value is not None:
        return
    left_idx = X[:, node.feature] <= node.threshold
    right_idx = X[:, node.feature] > node.threshold
    store_samples(node.left, X[left_idx], y[left_idx])
    store_samples(node.right, X[right_idx], y[right_idx])

store_samples(full_tree, X_train.values, y_train)


post_prune(full_tree, X_val, y_val)


## Evaluation
- Train using the training set.
- Tune depth and pruning using validation set.
- Report final results on test set.
- Metrics to report:
    - Accuracy
    - Precision
    - Recall
    - F1-score
    - Confusion Matrix
- Compare your implementation with sklearn.tree.DecisionTreeClassifier.

In [25]:


full_tree = build_tree(X_train.values, y_train, max_depth=None, criterion='entropy')


store_samples(full_tree, X_train.values, y_train)


In [26]:
best_depth = None
best_val_acc = 0

for depth in range(1, 16):
    tree = build_tree(X_train.values, y_train, max_depth=depth, criterion='entropy')
    y_val_pred = predict_tree(tree, X_val.values)
    val_acc = np.mean(y_val_pred == y_val)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_depth = depth

print(f"Best depth: {best_depth}, Validation Accuracy: {best_val_acc:.4f}")


Best depth: 7, Validation Accuracy: 0.5838


In [27]:
# Grow tree with best depth
tree_best = build_tree(X_train.values, y_train, max_depth=best_depth, criterion='entropy')
store_samples(tree_best, X_train.values, y_train)

# Prune using validation set
post_prune(tree_best, X_val, y_val)


In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Predictions
y_test_pred = predict_tree(tree_best, X_test.values)

# Metrics
acc = accuracy_score(y_test, y_test_pred)
prec = precision_score(y_test, y_test_pred, average='weighted')
rec = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
cm = confusion_matrix(y_test, y_test_pred)

print("Custom Tree Metrics on Test Set")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1-score: {f1:.4f}")
print("Confusion Matrix:\n", cm)


Custom Tree Metrics on Test Set
Accuracy: 0.5799
Precision: 0.3928
Recall: 0.5799
F1-score: 0.4642
Confusion Matrix:
 [[4722    0  222    0]
 [2153    0  119    0]
 [ 766    1  802    0]
 [ 341    1  398    0]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Metrics
acc_sk = accuracy_score(y_test, y_test_pred_sk)
prec_sk = precision_score(y_test, y_test_pred_sk, average='weighted')
rec_sk = recall_score(y_test, y_test_pred_sk, average='weighted')
f1_sk = f1_score(y_test, y_test_pred_sk, average='weighted')
cm_sk = confusion_matrix(y_test, y_test_pred_sk)

print("\nSklearn Tree Metrics on Test Set")
print(f"Accuracy: {acc_sk:.4f}")
print(f"Precision: {prec_sk:.4f}")
print(f"Recall: {rec_sk:.4f}")
print(f"F1-score: {f1_sk:.4f}")
print("Confusion Matrix:\n", cm_sk)



Sklearn Tree Metrics on Test Set
Accuracy: 0.5772
Precision: 0.4616
Recall: 0.5772
F1-score: 0.4648
Confusion Matrix:
 [[4686    5  250    3]
 [2128    3  141    0]
 [ 738    3  802   26]
 [ 327    2  404    7]]


## Repeating the experiment with Gini

In [32]:

tree = build_tree(X_train.values, y_train, max_depth=2, criterion='gini')
y_pred = predict_tree(tree, X_test.values)


print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.5639895013123359


In [33]:

tree = build_tree(X_train.values, y_train, max_depth=4, criterion='gini')
y_pred = predict_tree(tree, X_test.values)


print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.5742782152230971


In [34]:

tree = build_tree(X_train.values, y_train, max_depth=6, criterion='gini')
y_pred = predict_tree(tree, X_test.values)


print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.579002624671916
